<a href="https://colab.research.google.com/github/chsuknapat/lr-crops/blob/main/AIBuilder_Project_F1.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install prophet
!pip install plotly==4.14.3
!pip install -U scikit-learn
!pip install pmdarima
!pip install statsmodels

In [3]:
import pandas as pd
import numpy as np
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn import metrics

# Rain Prediction

In [ ]:
df = pd.read_csv('Rain_2020_2015_clean.csv',thousands=',')
df=df.drop(columns=['CODE', 'ZONE'])
df['RAIN'] =pd.to_numeric(df['RAIN'])
df['DATE'] =pd.to_datetime(df['DATE'], format = '%Y/%m/%d')
province_out = ['OtherN','OtherNE','OtherC','OtherS','OtherNNE','OtherNC','OtherNS','OtherNEC','OtherNES','OtherCS','OtherNNEC','OtherNNES','OtherNCS','OtherNECS','WholeKingdom']
df = df[~df['PROVINCE'].isin(province_out)] 
df=df.rename(columns={'DATE' : 'ds', 'RAIN': 'y'})

df_rain = pd.DataFrame()
grouped = df.groupby('PROVINCE')

for g in grouped.groups:
     group = grouped.get_group(g)
     m = Prophet()
     m.fit(group)
     future = m.make_future_dataframe(periods=24,freq='MS')
     forecast = m.predict(future)
     forecast_rain = forecast [['ds','yhat','yhat_lower','yhat_upper']]
     forecast_rain['PROVINCE'] = g
     df_rain = pd.concat([df_rain,forecast_rain],ignore_index=True)


In [ ]:
df_rain=df_rain[['PROVINCE','ds','yhat','yhat_lower','yhat_upper']]
df_rain.to_csv('df_rain.csv', index=False)
df_rain


,PROVINCE,ds,yhat,yhat_lower,yhat_upper
0,AmnatChareon,2015-01-01,-8.452468,-96.691128,81.225284
1,AmnatChareon,2015-02-01,-11.143993,-95.025521,79.939086
2,AmnatChareon,2015-03-01,9.317927,-84.591898,98.033734
3,AmnatChareon,2015-04-01,40.047984,-48.314823,127.630758
4,AmnatChareon,2015-05-01,132.308874,42.349677,222.401476
...,...,...,...,...,...
7291,Yasothon,2022-08-01,354.001609,254.450240,450.895050
7292,Yasothon,2022-09-01,170.932106,73.315919,269.206461
7293,Yasothon,2022-10-01,93.949718,-7.177807,194.783461
7294,Yasothon,2022-11-01,26.105843,-65.952463,127.359961


In [ ]:
plot_plotly(m, forecast)

In [ ]:
plot_components_plotly(m, forecast)

# Temp Prediction

In [ ]:
df = pd.read_csv('Temp_2020_2015_clean.csv',thousands=',')
df= df.drop(columns=['CODE','ZONE '])
df['TEMP'] =pd.to_numeric(df['TEMP'])
df['DATE'] =pd.to_datetime(df['DATE'], format = '%Y/%m/%d')
province_out = ['Bueng Kan']
df = df[~df['PROVINCE'].isin(province_out)]
df=df.rename(columns={'DATE' : 'ds', 'TEMP': 'y'})

df_temp = pd.DataFrame()
grouped = df.groupby('PROVINCE')

for g in grouped.groups:
     group = grouped.get_group(g)
     m = Prophet()
     m.fit(group)
     future = m.make_future_dataframe(periods=24,freq='MS')
     forecast = m.predict(future)
     forecast_temp = forecast [['ds','yhat','yhat_lower','yhat_upper']]
     forecast_temp['PROVINCE'] = g
     df_temp = pd.concat([df_temp,forecast_temp],ignore_index=True)
     #print(df_temp)

In [ ]:
df_temp=df_temp[['PROVINCE','ds','yhat','yhat_lower','yhat_upper']]
df_temp.to_csv('df_temp.csv', index=False)
df_temp

,PROVINCE,ds,yhat,yhat_lower,yhat_upper
0,AmnatChareon,2015-01-01,25.532592,24.234456,26.930437
1,AmnatChareon,2015-02-01,26.123221,24.843292,27.515373
2,AmnatChareon,2015-03-01,29.499186,28.153893,30.849063
3,AmnatChareon,2015-04-01,29.951251,28.546904,31.270188
4,AmnatChareon,2015-05-01,29.911131,28.631581,31.221803
...,...,...,...,...,...
7291,Yasothon,2022-08-01,27.814423,26.377241,29.266911
7292,Yasothon,2022-09-01,28.112954,26.733247,29.398491
7293,Yasothon,2022-10-01,27.336890,25.868784,28.755239
7294,Yasothon,2022-11-01,26.720270,25.384164,28.133680


In [ ]:
plot_plotly(m, forecast)

In [ ]:
plot_components_plotly(m, forecast)

# All Rain Temp

In [4]:
df_rain = pd.read_csv('/content/df_rain.csv')
df_rain['YEAR'] = df_rain['ds'].apply(lambda x:x[:4])
df_rain['MONTH'] = df_rain['ds'].apply(lambda x:x[5:7])
df_rain = pd.DataFrame(df_rain.pivot_table(index=['PROVINCE','YEAR'], columns = 'MONTH' , values = 'yhat' )).reset_index()

df_temp = pd.read_csv('/content/df_temp.csv')
df_temp['YEAR'] = df_temp['ds'].apply(lambda x:x[:4])
df_temp['MONTH'] =df_temp['ds'].apply(lambda x:x[5:7])
df_temp = pd.DataFrame(df_temp.pivot_table(index=['PROVINCE','YEAR'], columns = 'MONTH' , values = 'yhat' )).reset_index()

left =  df_rain.set_index(['PROVINCE' , 'YEAR' ])
right = df_temp.set_index(['PROVINCE' , 'YEAR' ])
df_rt = left.join(right, lsuffix='_rain' , rsuffix='_temp').reset_index()
df_rt['YEAR'] = pd.to_datetime(df_rt['YEAR'], format= '%Y').dt.to_period('Y')
df_rt


MONTH,PROVINCE,YEAR,01_rain,02_rain,03_rain,04_rain,05_rain,06_rain,07_rain,08_rain,09_rain,10_rain,11_rain,12_rain,01_temp,02_temp,03_temp,04_temp,05_temp,06_temp,07_temp,08_temp,09_temp,10_temp,11_temp,12_temp
0,AmnatChareon,2015,-8.452468,-11.143993,9.317927,40.047984,132.308874,83.638318,276.665117,163.106750,248.656722,17.320519,-9.417855,-14.518534,25.532592,26.123221,29.499186,29.951251,29.911131,29.534026,27.467133,27.961936,27.458547,27.765355,26.917975,25.586040
1,AmnatChareon,2016,-0.149627,-1.197657,4.825531,64.515513,85.982813,191.816058,261.777505,162.422592,138.393380,100.429790,14.556256,-2.341514,25.128755,26.104537,29.382494,30.298239,29.608400,28.612887,28.045367,28.220324,28.126634,27.151423,26.727753,24.651856
2,AmnatChareon,2017,-5.166075,-11.602064,12.895441,62.776092,108.243195,162.263165,272.604811,168.574599,181.974547,79.779704,12.817424,-0.237369,26.295526,26.144813,29.411813,30.170891,29.695182,28.914633,27.849315,28.125565,27.889394,27.341102,26.780784,24.961059
3,AmnatChareon,2018,2.712362,-1.776458,20.813654,61.038896,130.107234,132.611768,283.986823,175.226619,225.161368,58.487822,11.239838,2.121305,25.903071,26.118474,29.442510,30.047648,29.789239,29.215653,27.651103,28.034718,27.662629,27.542469,26.841573,25.270414
4,AmnatChareon,2019,10.805284,8.113759,28.575679,59.305736,151.566626,102.896070,295.922869,182.364502,267.914474,36.578271,9.839897,4.739218,25.512470,26.103875,29.480540,29.933382,29.894012,29.517683,27.451541,27.947121,27.444508,27.752066,26.905462,25.574279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603,Yasothon,2018,9.230865,6.351706,20.088082,44.073648,88.028297,89.780464,202.545408,337.586070,154.516567,77.534179,9.690304,2.651754,25.332139,26.859359,29.151138,30.448031,29.717792,28.668614,28.115725,27.814359,28.112890,27.336825,26.720206,25.293709
604,Yasothon,2019,19.772295,8.839932,23.383063,59.125755,96.400649,64.007791,192.018433,392.690677,153.658319,70.151751,6.052871,7.400603,25.231656,26.561478,29.174981,30.492049,29.827532,28.834916,28.035924,27.643020,27.984359,27.458222,26.738853,25.612884
605,Yasothon,2020,30.371292,11.301082,30.267109,30.195572,85.925780,157.364834,243.256688,244.308565,170.274345,108.017229,33.818269,10.360734,25.122308,26.263445,29.119424,30.369158,29.501866,28.325078,28.257614,28.153709,28.383227,27.107375,26.691916,24.644792
606,Yasothon,2021,15.169560,20.252176,33.325512,45.369170,95.480656,131.855594,230.571933,299.051260,170.996984,101.105531,29.893825,14.582337,25.422972,27.156405,29.132650,30.407073,29.609215,28.498640,28.189700,27.984721,28.245954,27.219883,26.704558,24.970936


# Rice

In [ ]:
df_rice = pd.read_csv('/content/Rice_2019_2010_clean.csv',thousands=',')
df_rice= df_rice.drop(columns=['CODE','ZONE','AREA-R','YIELD-T'])
df_rice= df_rice[df_rice['PROVINCE'] != 'Bueng Kan' ]
df_rice['YIELD-KG/R'] = pd.to_numeric(df_rice['YIELD-KG/R'])
df_rice['YEAR'] =  pd.to_datetime(df_rice['YEAR'], format= '%Y').dt.to_period('Y')
df_rice['CROP'] = 'Rice'

left =  df_rice.set_index(['PROVINCE' , 'YEAR' ])
right = df_rt.set_index(['PROVINCE' , 'YEAR' ])
df_rice = left.join(right, lsuffix=' ' , rsuffix=' ').dropna().reset_index()
df_rice['PROVINCE_CROP'] = df_rice['PROVINCE']+'_rice'
df_rice.sort_values(['PROVINCE','YEAR'])

df_riceclust = df_rice.groupby(['PROVINCE_CROP']).mean().reset_index()
x =df_riceclust.drop(columns=['PROVINCE_CROP','YIELD-KG/R']) 
seed = 42
model = KMeans(n_clusters=4,random_state = seed)
kmeans = model.fit_predict(x)

df_riceclust['CLUSTER'] = kmeans
df_riceclust['CLUSTER'] =df_riceclust['CLUSTER'].astype(str)+'_rice'
df_riceclust =df_riceclust[['PROVINCE_CROP','CLUSTER']]

left = df_rice.set_index(['PROVINCE_CROP'])
right = df_riceclust.set_index(['PROVINCE_CROP'])
df_rice = left.join(right, lsuffix=' ', rsuffix=' ').reset_index()
df_rice

,PROVINCE_CROP,PROVINCE,YEAR,YIELD-KG/R,CROP,01_rain,02_rain,03_rain,04_rain,05_rain,06_rain,07_rain,08_rain,09_rain,10_rain,11_rain,12_rain,01_temp,02_temp,03_temp,04_temp,05_temp,06_temp,07_temp,08_temp,09_temp,10_temp,11_temp,12_temp,CLUSTER
0,AmnatChareon_rice,AmnatChareon,2015,335,Rice,-8.452468,-11.143993,9.317927,40.047984,132.308874,83.638318,276.665117,163.106750,248.656722,17.320519,-9.417855,-14.518534,25.532592,26.123221,29.499186,29.951251,29.911131,29.534026,27.467133,27.961936,27.458547,27.765355,26.917975,25.586040,0_rice
1,AmnatChareon_rice,AmnatChareon,2016,334,Rice,-0.149627,-1.197657,4.825531,64.515513,85.982813,191.816058,261.777505,162.422592,138.393380,100.429790,14.556256,-2.341514,25.128755,26.104537,29.382494,30.298239,29.608400,28.612887,28.045367,28.220324,28.126634,27.151423,26.727753,24.651856,0_rice
2,AmnatChareon_rice,AmnatChareon,2017,335,Rice,-5.166075,-11.602064,12.895441,62.776092,108.243195,162.263165,272.604811,168.574599,181.974547,79.779704,12.817424,-0.237369,26.295526,26.144813,29.411813,30.170891,29.695182,28.914633,27.849315,28.125565,27.889394,27.341102,26.780784,24.961059,0_rice
3,AmnatChareon_rice,AmnatChareon,2018,360,Rice,2.712362,-1.776458,20.813654,61.038896,130.107234,132.611768,283.986823,175.226619,225.161368,58.487822,11.239838,2.121305,25.903071,26.118474,29.442510,30.047648,29.789239,29.215653,27.651103,28.034718,27.662629,27.542469,26.841573,25.270414,0_rice
4,AmnatChareon_rice,AmnatChareon,2019,320,Rice,10.805284,8.113759,28.575679,59.305736,151.566626,102.896070,295.922869,182.364502,267.914474,36.578271,9.839897,4.739218,25.512470,26.103875,29.480540,29.933382,29.894012,29.517683,27.451541,27.947121,27.444508,27.752066,26.905462,25.574279,0_rice
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,Yasothon_rice,Yasothon,2015,338,Rice,3.356757,-7.575607,6.967524,42.710216,79.985110,47.592252,175.602894,376.275138,137.242780,53.736212,-10.362668,-9.014936,25.247144,26.576479,29.189541,30.506123,29.841134,28.848031,28.048568,27.655178,27.996029,27.469421,26.749565,25.623125,0_rice
356,Yasothon_rice,Yasothon,2016,341,Rice,13.955753,-5.114456,13.851570,13.780033,69.510242,140.949295,226.841149,227.893026,153.858806,91.601690,17.402730,-6.054805,25.132061,26.272712,29.128235,30.377482,29.509719,28.332444,28.264509,28.160116,28.389148,27.112825,26.696878,24.649283,0_rice
357,Yasothon_rice,Yasothon,2017,312,Rice,-1.245979,3.836638,16.909974,28.953632,79.065118,115.440055,214.156394,282.635722,154.581445,84.689993,13.478286,-1.833202,25.426977,27.159923,29.135728,30.409663,29.611337,28.500278,28.190981,27.985633,28.246709,27.220485,26.705002,24.971253,0_rice
358,Yasothon_rice,Yasothon,2018,345,Rice,9.230865,6.351706,20.088082,44.073648,88.028297,89.780464,202.545408,337.586070,154.516567,77.534179,9.690304,2.651754,25.332139,26.859359,29.151138,30.448031,29.717792,28.668614,28.115725,27.814359,28.112890,27.336825,26.720206,25.293709,0_rice


# Pineapple

In [ ]:
df_pine = pd.read_csv('/content/Pineapple_2019_2010_clean.csv',thousands=',')
df_pine= df_pine.drop(columns=['CODE','ZONE','AREA-R','YIELD-T'])
df_pine= df_pine[df_pine['PROVINCE'] != 'Bueng Kan' ]
df_pine['YIELD-KG/R'] = pd.to_numeric(df_pine['YIELD-KG/R'])
df_pine['YEAR'] =  pd.to_datetime(df_pine['YEAR'], format= '%Y').dt.to_period('Y')
df_pine['CROP'] = 'Pineapple'

left =  df_pine.set_index(['PROVINCE' , 'YEAR' ])
right = df_rt.set_index(['PROVINCE' , 'YEAR' ])
df_pine = left.join(right, lsuffix=' ' , rsuffix=' ').dropna().reset_index()
df_pine['PROVINCE_CROP'] = df_pine['PROVINCE']+'_pine'
df_pine.sort_values(['PROVINCE','YEAR'])

df_pineclust = df_pine.groupby(['PROVINCE_CROP']).mean().reset_index()
x = df_pineclust.drop(columns=['PROVINCE_CROP','YIELD-KG/R']) 
seed = 42
model = KMeans(n_clusters=4,random_state = seed)
kmeans = model.fit_predict(x)

df_pineclust['CLUSTER'] = kmeans
df_pineclust['CLUSTER'] = df_pineclust['CLUSTER'].astype(str)+'_pine'
df_pineclust = df_pineclust[['PROVINCE_CROP','CLUSTER']]

left = df_pine.set_index(['PROVINCE_CROP'])
right = df_pineclust.set_index(['PROVINCE_CROP'])
df_pine = left.join(right, lsuffix=' ', rsuffix=' ').reset_index()
df_pine                                                                                               


,PROVINCE_CROP,PROVINCE,YEAR,YIELD-KG/R,CROP,01_rain,02_rain,03_rain,04_rain,05_rain,06_rain,07_rain,08_rain,09_rain,10_rain,11_rain,12_rain,01_temp,02_temp,03_temp,04_temp,05_temp,06_temp,07_temp,08_temp,09_temp,10_temp,11_temp,12_temp,CLUSTER
0,Chachoengsao_pine,Chachoengsao,2015,4352,Pineapple,36.554880,25.192649,64.511225,108.184841,217.013576,144.447315,170.811859,111.659414,280.816776,168.520200,75.966135,13.977254,25.811819,26.905821,27.702191,28.699026,28.812390,28.245228,28.656463,27.960630,27.616813,27.632967,27.131294,26.433513,0_pine
1,Chachoengsao_pine,Chachoengsao,2016,4286,Pineapple,14.196086,33.718222,69.277818,102.884625,190.551568,164.972635,175.613011,157.867837,230.045067,196.344061,36.980749,-0.523812,27.058606,28.071341,29.207172,30.124359,29.891557,28.233914,28.055500,28.501116,28.336286,27.674048,27.496189,26.349920,0_pine
2,Chachoengsao_pine,Chachoengsao,2017,4578,Pineapple,84.926166,12.825464,68.895459,105.787477,200.825417,159.602777,175.355747,143.415115,248.072856,188.319069,51.149895,6.009608,25.840767,26.928548,28.477887,29.346639,29.156323,27.795020,27.761238,27.785968,27.509418,27.026825,26.694476,25.648032,0_pine
3,Chachoengsao_pine,Chachoengsao,2018,5084,Pineapple,62.638251,20.725546,68.551096,108.798857,210.890832,153.999203,174.992596,129.247398,266.240653,180.282398,65.388244,12.082636,25.475962,26.504659,27.616700,28.548252,28.508504,27.547216,27.745242,27.414986,27.104095,26.872247,26.457335,25.584841,0_pine
4,Chachoengsao_pine,Chachoengsao,2019,4980,Pineapple,40.292036,28.928588,68.246066,111.918464,220.746023,148.178545,174.541911,115.388250,284.544395,172.246641,79.691360,17.701301,25.816222,26.844971,27.582347,28.513866,28.564022,27.931476,28.279437,27.518029,27.108645,27.060986,26.493415,25.731861,0_pine
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,Uttaradit_pine,Uttaradit,2015,3226,Pineapple,8.810148,-3.069222,12.686231,46.845285,85.301811,94.317595,123.701372,209.290012,111.404468,95.418437,29.281316,15.378585,25.479290,26.562975,29.898435,31.026296,30.636747,29.939410,28.261355,28.512897,28.579176,28.454133,27.814507,25.122331,0_pine
125,Uttaradit_pine,Uttaradit,2016,3265,Pineapple,-0.039358,-5.187432,-4.485985,34.424878,134.230265,62.310777,112.531066,213.967314,168.939370,72.958866,1.594496,4.764366,24.935491,26.791672,29.637115,31.327784,30.807620,29.314132,28.601705,28.529525,28.807684,28.042275,27.355196,24.888137,0_pine
126,Uttaradit_pine,Uttaradit,2017,3500,Pineapple,34.156561,9.860035,3.843067,41.483869,120.549489,75.439552,119.157112,215.475243,152.356186,82.748965,13.517028,11.327165,26.433712,25.989598,29.681427,31.183104,30.700918,29.476218,28.443788,28.476630,28.682419,28.130700,27.465025,24.928227,0_pine
127,Uttaradit_pine,Uttaradit,2018,3441,Pineapple,25.629393,7.365476,12.278078,48.333655,106.943842,88.820259,125.591489,216.622750,135.879096,92.946043,25.459075,17.575052,25.886503,26.203491,29.723973,31.038593,30.600439,29.663452,28.330572,28.493569,28.650882,28.334136,27.706440,25.115940,0_pine


# Cassava

In [ ]:
df_cas = pd.read_csv('/content/Cassava_2020_2011_clean.csv',thousands=',')
df_cas= df_cas.drop(columns=['CODE','ZONE','AREA-R','YIELD-T'])
df_cas= df_cas[df_cas['PROVINCE'] != 'Bueng Kan' ]
df_cas['YIELD-KG/R'] = pd.to_numeric(df_cas['YIELD-KG/R'])
df_cas['YEAR'] =  pd.to_datetime(df_cas['YEAR'], format= '%Y').dt.to_period('Y')
df_cas['CROP'] = 'Cassava'

left =  df_cas.set_index(['PROVINCE' , 'YEAR' ])
right = df_rt.set_index(['PROVINCE' , 'YEAR' ])
df_cas = left.join(right, lsuffix=' ' , rsuffix=' ').dropna().reset_index()
df_cas['PROVINCE_CROP'] = df_cas['PROVINCE']+'_cass'
df_cas.sort_values(['PROVINCE','YEAR'])

df_casclust = df_cas.groupby(['PROVINCE_CROP']).mean().reset_index()
x =df_casclust.drop(columns=['PROVINCE_CROP','YIELD-KG/R']) 
seed = 42
model = KMeans(n_clusters=4,random_state = seed)
kmeans = model.fit_predict(x)

df_casclust['CLUSTER'] = kmeans
df_casclust['CLUSTER'] =df_casclust['CLUSTER'].astype(str)+'_cass'
df_casclust =df_casclust[['PROVINCE_CROP','CLUSTER']]

left = df_cas.set_index(['PROVINCE_CROP'])
right = df_casclust.set_index(['PROVINCE_CROP'])
df_cas = left.join(right, lsuffix=' ', rsuffix=' ').reset_index()
df_cas

,PROVINCE_CROP,PROVINCE,YEAR,YIELD-KG/R,CROP,01_rain,02_rain,03_rain,04_rain,05_rain,06_rain,07_rain,08_rain,09_rain,10_rain,11_rain,12_rain,01_temp,02_temp,03_temp,04_temp,05_temp,06_temp,07_temp,08_temp,09_temp,10_temp,11_temp,12_temp,CLUSTER
0,AmnatChareon_cass,AmnatChareon,2015,3353,Cassava,-8.452468,-11.143993,9.317927,40.047984,132.308874,83.638318,276.665117,163.106750,248.656722,17.320519,-9.417855,-14.518534,25.532592,26.123221,29.499186,29.951251,29.911131,29.534026,27.467133,27.961936,27.458547,27.765355,26.917975,25.586040,3_cass
1,AmnatChareon_cass,AmnatChareon,2016,3406,Cassava,-0.149627,-1.197657,4.825531,64.515513,85.982813,191.816058,261.777505,162.422592,138.393380,100.429790,14.556256,-2.341514,25.128755,26.104537,29.382494,30.298239,29.608400,28.612887,28.045367,28.220324,28.126634,27.151423,26.727753,24.651856,3_cass
2,AmnatChareon_cass,AmnatChareon,2017,3593,Cassava,-5.166075,-11.602064,12.895441,62.776092,108.243195,162.263165,272.604811,168.574599,181.974547,79.779704,12.817424,-0.237369,26.295526,26.144813,29.411813,30.170891,29.695182,28.914633,27.849315,28.125565,27.889394,27.341102,26.780784,24.961059,3_cass
3,AmnatChareon_cass,AmnatChareon,2018,3565,Cassava,2.712362,-1.776458,20.813654,61.038896,130.107234,132.611768,283.986823,175.226619,225.161368,58.487822,11.239838,2.121305,25.903071,26.118474,29.442510,30.047648,29.789239,29.215653,27.651103,28.034718,27.662629,27.542469,26.841573,25.270414,3_cass
4,AmnatChareon_cass,AmnatChareon,2019,3501,Cassava,10.805284,8.113759,28.575679,59.305736,151.566626,102.896070,295.922869,182.364502,267.914474,36.578271,9.839897,4.739218,25.512470,26.103875,29.480540,29.933382,29.894012,29.517683,27.451541,27.947121,27.444508,27.752066,26.905462,25.574279,3_cass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,Yasothon_cass,Yasothon,2016,3551,Cassava,13.955753,-5.114456,13.851570,13.780033,69.510242,140.949295,226.841149,227.893026,153.858806,91.601690,17.402730,-6.054805,25.132061,26.272712,29.128235,30.377482,29.509719,28.332444,28.264509,28.160116,28.389148,27.112825,26.696878,24.649283,0_cass
274,Yasothon_cass,Yasothon,2017,3688,Cassava,-1.245979,3.836638,16.909974,28.953632,79.065118,115.440055,214.156394,282.635722,154.581445,84.689993,13.478286,-1.833202,25.426977,27.159923,29.135728,30.409663,29.611337,28.500278,28.190981,27.985633,28.246709,27.220485,26.705002,24.971253,0_cass
275,Yasothon_cass,Yasothon,2018,3652,Cassava,9.230865,6.351706,20.088082,44.073648,88.028297,89.780464,202.545408,337.586070,154.516567,77.534179,9.690304,2.651754,25.332139,26.859359,29.151138,30.448031,29.717792,28.668614,28.115725,27.814359,28.112890,27.336825,26.720206,25.293709,0_cass
276,Yasothon_cass,Yasothon,2019,3695,Cassava,19.772295,8.839932,23.383063,59.125755,96.400649,64.007791,192.018433,392.690677,153.658319,70.151751,6.052871,7.400603,25.231656,26.561478,29.174981,30.492049,29.827532,28.834916,28.035924,27.643020,27.984359,27.458222,26.738853,25.612884,0_cass


# Maize

In [ ]:
df_maize = pd.read_csv('/content/Maize_2019_2010_clean.csv',thousands=',')
df_maize= df_maize.drop(columns=['CODE','ZONE','AREA-R','YIELD-T'])
df_maize= df_maize[df_maize['PROVINCE'] != 'Bueng Kan' ]
df_maize['YIELD-KG/R'] = pd.to_numeric(df_maize['YIELD-KG/R'])
df_maize['YEAR'] =  pd.to_datetime(df_maize['YEAR'], format= '%Y').dt.to_period('Y')
df_maize['CROP'] = 'Maize'

left =  df_maize.set_index(['PROVINCE' , 'YEAR' ])
right = df_rt.set_index(['PROVINCE' , 'YEAR' ])
df_maize = left.join(right, lsuffix=' ' , rsuffix=' ').dropna().reset_index()
df_maize['PROVINCE_CROP'] = df_maize['PROVINCE']+'_maze'
df_maize.sort_values(['PROVINCE','YEAR'])

df_maizeclust = df_maize.groupby(['PROVINCE_CROP']).mean().reset_index()
x =df_maizeclust.drop(columns=['PROVINCE_CROP','YIELD-KG/R']) 
seed = 42
model = KMeans(n_clusters=4,random_state = seed)
kmeans = model.fit_predict(x)

df_maizeclust['CLUSTER'] = kmeans
df_maizeclust['CLUSTER'] =df_maizeclust['CLUSTER'].astype(str)+'_maze'
df_maizeclust =df_maizeclust[['PROVINCE_CROP','CLUSTER']]

left = df_maize.set_index(['PROVINCE_CROP'])
right =df_maizeclust.set_index(['PROVINCE_CROP'])
df_maize = left.join(right, lsuffix=' ', rsuffix=' ').reset_index()
df_maize

,PROVINCE_CROP,PROVINCE,YEAR,YIELD-KG/R,CROP,01_rain,02_rain,03_rain,04_rain,05_rain,06_rain,07_rain,08_rain,09_rain,10_rain,11_rain,12_rain,01_temp,02_temp,03_temp,04_temp,05_temp,06_temp,07_temp,08_temp,09_temp,10_temp,11_temp,12_temp,CLUSTER
0,AmnatChareon_maze,AmnatChareon,2018,273,Maize,2.712362,-1.776458,20.813654,61.038896,130.107234,132.611768,283.986823,175.226619,225.161368,58.487822,11.239838,2.121305,25.903071,26.118474,29.442510,30.047648,29.789239,29.215653,27.651103,28.034718,27.662629,27.542469,26.841573,25.270414,2_maze
1,AmnatChareon_maze,AmnatChareon,2019,522,Maize,10.805284,8.113759,28.575679,59.305736,151.566626,102.896070,295.922869,182.364502,267.914474,36.578271,9.839897,4.739218,25.512470,26.103875,29.480540,29.933382,29.894012,29.517683,27.451541,27.947121,27.444508,27.752066,26.905462,25.574279,2_maze
2,BuriRum_maze,BuriRum,2018,365,Maize,4.703562,9.791590,47.871254,80.897689,115.333175,99.766315,136.968665,180.627502,233.429558,110.700715,25.522758,4.306154,25.172023,26.545801,29.260178,29.704129,29.330444,28.290987,28.478206,27.515937,27.448703,26.659494,25.599577,24.837195,0_maze
3,BuriRum_maze,BuriRum,2019,525,Maize,10.447524,10.623976,49.270896,95.876699,116.117177,99.148147,123.108027,197.834537,253.544337,103.645828,23.693576,9.932726,25.118441,26.440650,29.401259,29.532863,29.323877,28.436817,28.805367,27.378310,27.262694,26.767590,25.492421,25.228769,0_maze
4,Chachoengsao_maze,Chachoengsao,2015,659,Maize,36.554880,25.192649,64.511225,108.184841,217.013576,144.447315,170.811859,111.659414,280.816776,168.520200,75.966135,13.977254,25.811819,26.905821,27.702191,28.699026,28.812390,28.245228,28.656463,27.960630,27.616813,27.632967,27.131294,26.433513,0_maze
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,Uttaradit_maze,Uttaradit,2017,690,Maize,34.156561,9.860035,3.843067,41.483869,120.549489,75.439552,119.157112,215.475243,152.356186,82.748965,13.517028,11.327165,26.433712,25.989598,29.681427,31.183104,30.700918,29.476218,28.443788,28.476630,28.682419,28.130700,27.465025,24.928227,0_maze
189,Uttaradit_maze,Uttaradit,2018,700,Maize,25.629393,7.365476,12.278078,48.333655,106.943842,88.820259,125.591489,216.622750,135.879096,92.946043,25.459075,17.575052,25.886503,26.203491,29.723973,31.038593,30.600439,29.663452,28.330572,28.493569,28.650882,28.334136,27.706440,25.115940,0_maze
190,Uttaradit_maze,Uttaradit,2019,651,Maize,16.934868,5.055531,20.811014,54.970102,93.426660,102.442478,131.826287,217.414960,119.529450,103.543451,37.406364,23.503664,25.518724,26.624837,29.980553,31.130842,30.762997,30.088088,28.431737,28.705706,28.794413,28.691074,28.073876,25.403404,0_maze
191,Yasothon_maze,Yasothon,2018,537,Maize,9.230865,6.351706,20.088082,44.073648,88.028297,89.780464,202.545408,337.586070,154.516567,77.534179,9.690304,2.651754,25.332139,26.859359,29.151138,30.448031,29.717792,28.668614,28.115725,27.814359,28.112890,27.336825,26.720206,25.293709,2_maze


# All Crop

In [ ]:
df_all = pd.concat([df_rice,df_pine,df_cas,df_maize], axis=0).reset_index(drop=True)#???
df_all.to_csv('df_all.csv' , index=False)
df_all

,PROVINCE_CROP,PROVINCE,YEAR,YIELD-KG/R,CROP,01_rain,02_rain,03_rain,04_rain,05_rain,06_rain,07_rain,08_rain,09_rain,10_rain,11_rain,12_rain,01_temp,02_temp,03_temp,04_temp,05_temp,06_temp,07_temp,08_temp,09_temp,10_temp,11_temp,12_temp,CLUSTER
0,AmnatChareon_rice,AmnatChareon,2015,335,Rice,-8.452468,-11.143993,9.317927,40.047984,132.308874,83.638318,276.665117,163.106750,248.656722,17.320519,-9.417855,-14.518534,25.532592,26.123221,29.499186,29.951251,29.911131,29.534026,27.467133,27.961936,27.458547,27.765355,26.917975,25.586040,0_rice
1,AmnatChareon_rice,AmnatChareon,2016,334,Rice,-0.149627,-1.197657,4.825531,64.515513,85.982813,191.816058,261.777505,162.422592,138.393380,100.429790,14.556256,-2.341514,25.128755,26.104537,29.382494,30.298239,29.608400,28.612887,28.045367,28.220324,28.126634,27.151423,26.727753,24.651856,0_rice
2,AmnatChareon_rice,AmnatChareon,2017,335,Rice,-5.166075,-11.602064,12.895441,62.776092,108.243195,162.263165,272.604811,168.574599,181.974547,79.779704,12.817424,-0.237369,26.295526,26.144813,29.411813,30.170891,29.695182,28.914633,27.849315,28.125565,27.889394,27.341102,26.780784,24.961059,0_rice
3,AmnatChareon_rice,AmnatChareon,2018,360,Rice,2.712362,-1.776458,20.813654,61.038896,130.107234,132.611768,283.986823,175.226619,225.161368,58.487822,11.239838,2.121305,25.903071,26.118474,29.442510,30.047648,29.789239,29.215653,27.651103,28.034718,27.662629,27.542469,26.841573,25.270414,0_rice
4,AmnatChareon_rice,AmnatChareon,2019,320,Rice,10.805284,8.113759,28.575679,59.305736,151.566626,102.896070,295.922869,182.364502,267.914474,36.578271,9.839897,4.739218,25.512470,26.103875,29.480540,29.933382,29.894012,29.517683,27.451541,27.947121,27.444508,27.752066,26.905462,25.574279,0_rice
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,Uttaradit_maze,Uttaradit,2017,690,Maize,34.156561,9.860035,3.843067,41.483869,120.549489,75.439552,119.157112,215.475243,152.356186,82.748965,13.517028,11.327165,26.433712,25.989598,29.681427,31.183104,30.700918,29.476218,28.443788,28.476630,28.682419,28.130700,27.465025,24.928227,0_maze
956,Uttaradit_maze,Uttaradit,2018,700,Maize,25.629393,7.365476,12.278078,48.333655,106.943842,88.820259,125.591489,216.622750,135.879096,92.946043,25.459075,17.575052,25.886503,26.203491,29.723973,31.038593,30.600439,29.663452,28.330572,28.493569,28.650882,28.334136,27.706440,25.115940,0_maze
957,Uttaradit_maze,Uttaradit,2019,651,Maize,16.934868,5.055531,20.811014,54.970102,93.426660,102.442478,131.826287,217.414960,119.529450,103.543451,37.406364,23.503664,25.518724,26.624837,29.980553,31.130842,30.762997,30.088088,28.431737,28.705706,28.794413,28.691074,28.073876,25.403404,0_maze
958,Yasothon_maze,Yasothon,2018,537,Maize,9.230865,6.351706,20.088082,44.073648,88.028297,89.780464,202.545408,337.586070,154.516567,77.534179,9.690304,2.651754,25.332139,26.859359,29.151138,30.448031,29.717792,28.668614,28.115725,27.814359,28.112890,27.336825,26.720206,25.293709,2_maze


# Regression By Cluster

In [ ]:
final_data =  pd.read_csv('/content/df_all.csv',thousands=',')
final_data_train = final_data[final_data['YEAR'] < 2018]
final_dic_cluster = {}
for i in final_data_train['CLUSTER'].unique():
    group = final_data_train[(final_data_train['CLUSTER'] == i )]
    x_train = group.drop(columns=['PROVINCE','YEAR','YIELD-KG/R','CROP', 'PROVINCE_CROP','CLUSTER'])
    y_train = group[['YIELD-KG/R']]
    coef_dic = {}
    model = LinearRegression()
    model.fit(x_train,y_train)
    coef = model.coef_
    intercept = model.intercept_
    coef_dic['01_rain'] = coef[0,0]
    coef_dic['02_rain'] = coef[0,1]
    coef_dic['03_rain'] = coef[0,2]
    coef_dic['04_rain'] = coef[0,3]
    coef_dic['05_rain'] = coef[0,4]
    coef_dic['06_rain'] = coef[0,5]
    coef_dic['07_rain'] = coef[0,6]
    coef_dic['08_rain'] = coef[0,7]
    coef_dic['09_rain'] = coef[0,8]
    coef_dic['10_rain'] = coef[0,9]
    coef_dic['11_rain'] = coef[0,10]
    coef_dic['12_rain'] = coef[0,11]
    coef_dic['01_temp'] = coef[0,12]
    coef_dic['02_temp'] = coef[0,13]
    coef_dic['03_temp'] = coef[0,14]
    coef_dic['04_temp'] = coef[0,15]
    coef_dic['05_temp'] = coef[0,16]
    coef_dic['06_temp'] = coef[0,17]
    coef_dic['07_temp'] = coef[0,18]
    coef_dic['08_temp'] = coef[0,19]
    coef_dic['09_temp'] = coef[0,20]
    coef_dic['10_temp'] = coef[0,21]
    coef_dic['11_temp'] = coef[0,22]
    coef_dic['12_temp'] = coef[0,23]
    coef_dic['intercept'] = intercept[0]
    final_dic_cluster[i] = coef_dic
print(final_dic_cluster)   

In [ ]:

final_data_test = final_data[final_data['YEAR'] > 2017]
final_data_test = final_data_test[final_data_test['CROP'] == 'Rice'].reset_index(drop = True)
y_test = np.array(final_data_test['YIELD-KG/R'])
y_predict = []
for i in range(len(final_data_test)):
    row = final_data_test.iloc[[i]]
    cluster = row['CLUSTER'].values[0]
    x_test = row.drop(['PROVINCE','YEAR','YIELD-KG/R','CROP','PROVINCE_CROP','CLUSTER'], axis=1)
    x_test = np.array(x_test)
    crop_coef = list(final_dic_cluster[cluster].values())
    crop_coef = crop_coef[:-1]
    crop_coef = np.array(crop_coef)
    crop_int = final_dic_cluster[cluster]['intercept']
    crop_yield = crop_int + np.sum(crop_coef*x_test)
    y_predict.append(crop_yield)
mape1 = metrics.mean_absolute_percentage_error(y_test,y_predict)*100
rmse1 = metrics.mean_squared_error(y_test,y_predict, squared=False)
print('LR by Cluster')
print('MAPE1:' ,mape1)
print('RMSE1:' ,rmse1)

LR by Cluster
MAPE1: 23.066261861159866
RMSE1: 156.1683075067733


# Regression By  PROVINCE

In [ ]:
final_data =  pd.read_csv('/content/df_all.csv',thousands=',')
final_data_train = final_data[final_data['YEAR'] < 2018]
final_dic_province = {}
for i in final_data_train['PROVINCE_CROP'].unique():
    group = final_data_train[(final_data_train['PROVINCE_CROP'] == i )]
    x_train = group.drop(columns=['PROVINCE','YEAR','YIELD-KG/R','CROP', 'PROVINCE_CROP','CLUSTER'])
    y_train = group[['YIELD-KG/R']]
    coef_dic = {}
    model = LinearRegression()
    model.fit(x_train,y_train)
    coef = model.coef_
    intercept = model.intercept_
    coef_dic['01_rain'] = coef[0,0]
    coef_dic['02_rain'] = coef[0,1]
    coef_dic['03_rain'] = coef[0,2]
    coef_dic['04_rain'] = coef[0,3]
    coef_dic['05_rain'] = coef[0,4]
    coef_dic['06_rain'] = coef[0,5]
    coef_dic['07_rain'] = coef[0,6]
    coef_dic['08_rain'] = coef[0,7]
    coef_dic['09_rain'] = coef[0,8]
    coef_dic['10_rain'] = coef[0,9]
    coef_dic['11_rain'] = coef[0,10]
    coef_dic['12_rain'] = coef[0,11]
    coef_dic['01_temp'] = coef[0,12]
    coef_dic['02_temp'] = coef[0,13]
    coef_dic['03_temp'] = coef[0,14]
    coef_dic['04_temp'] = coef[0,15]
    coef_dic['05_temp'] = coef[0,16]
    coef_dic['06_temp'] = coef[0,17]
    coef_dic['07_temp'] = coef[0,18]
    coef_dic['08_temp'] = coef[0,19]
    coef_dic['09_temp'] = coef[0,20]
    coef_dic['10_temp'] = coef[0,21]
    coef_dic['11_temp'] = coef[0,22]
    coef_dic['12_temp'] = coef[0,23]
    coef_dic['intercept'] = intercept[0]
    final_dic_province[i] = coef_dic
print(final_dic_province)   

In [ ]:

final_data_test = final_data[final_data['YEAR'] > 2017]
final_data_test = final_data_test[final_data_test['CROP'] == 'Rice'].reset_index(drop = True)
y_test = np.array(final_data_test['YIELD-KG/R'])
y_predict = []
for i in range(len(final_data_test)):
    row = final_data_test.iloc[[i]]
    province_crop = row['PROVINCE_CROP'].values[0]
    x_test = row.drop(['PROVINCE','YEAR','YIELD-KG/R','CROP','PROVINCE_CROP','CLUSTER'], axis=1)
    x_test = np.array(x_test)
    crop_coef = list(final_dic_province[province_crop].values())
    crop_coef = crop_coef[:-1]
    crop_coef = np.array(crop_coef)
    crop_int = final_dic_province[province_crop]['intercept']
    crop_yield = crop_int + np.sum(crop_coef*x_test)
    y_predict.append(crop_yield)

mape2 = metrics.mean_absolute_percentage_error(y_test,y_predict)*100
rmse2 = metrics.mean_squared_error(y_test,y_predict, squared=False)
print('LR by province')
print('MAPE2:' ,mape2)
print('RMSE2:' ,rmse2)

LR by province
MAPE2: 7.70303113687685
RMSE2: 43.53094873180956


# Forecast

In [7]:
final_data =  pd.read_csv('/content/df_all.csv',thousands=',')
final_dic_province = {}
for i in final_data['PROVINCE_CROP'].unique():
    group = final_data[(final_data['PROVINCE_CROP'] == i )]
    x = group.drop(columns=['PROVINCE','YEAR','YIELD-KG/R','CROP', 'PROVINCE_CROP','CLUSTER'])
    y = group[['YIELD-KG/R']]
    coef_dic = {}
    model = LinearRegression()
    model.fit(x,y)
    coef = model.coef_
    intercept = model.intercept_
    coef_dic['01_rain'] = coef[0,0]
    coef_dic['02_rain'] = coef[0,1]
    coef_dic['03_rain'] = coef[0,2]
    coef_dic['04_rain'] = coef[0,3]
    coef_dic['05_rain'] = coef[0,4]
    coef_dic['06_rain'] = coef[0,5]
    coef_dic['07_rain'] = coef[0,6]
    coef_dic['08_rain'] = coef[0,7]
    coef_dic['09_rain'] = coef[0,8]
    coef_dic['10_rain'] = coef[0,9]
    coef_dic['11_rain'] = coef[0,10]
    coef_dic['12_rain'] = coef[0,11]
    coef_dic['01_temp'] = coef[0,12]
    coef_dic['02_temp'] = coef[0,13]
    coef_dic['03_temp'] = coef[0,14]
    coef_dic['04_temp'] = coef[0,15]
    coef_dic['05_temp'] = coef[0,16]
    coef_dic['06_temp'] = coef[0,17]
    coef_dic['07_temp'] = coef[0,18]
    coef_dic['08_temp'] = coef[0,19]
    coef_dic['09_temp'] = coef[0,20]
    coef_dic['10_temp'] = coef[0,21]
    coef_dic['11_temp'] = coef[0,22]
    coef_dic['12_temp'] = coef[0,23]
    coef_dic['intercept'] = intercept[0]
    final_dic_province[i] = coef_dic
print(final_dic_province)   

{'AmnatChareon_rice': {'01_rain': -3.050224130314318, '02_rain': 6.318676781154064, '03_rain': 7.249467858482189, '04_rain': 2.693390982831628, '05_rain': 20.105093911669975, '06_rain': 5.580463272688475, '07_rain': -25.953174477228494, '08_rain': -30.552453060355816, '09_rain': 15.446568031280778, '10_rain': 23.13553664834005, '11_rain': -9.08713900321959, '12_rain': -12.667395274228092, '01_temp': -0.27683297078133645, '02_temp': -0.2829661553399279, '03_temp': 0.9347362698583234, '04_temp': -0.12422363074448023, '05_temp': -0.5078624967370403, '06_temp': -0.34397319073908433, '07_temp': 0.366564042683596, '08_temp': -1.4511301480252465, '09_temp': 0.21100864082379076, '10_temp': -1.9849483668401293, '11_temp': -0.14058921312840236, '12_temp': 0.6100493823652436, 'intercept': 4813.898106382516}, 'AngThong_rice': {'01_rain': 4.4618970748819455, '02_rain': -1.7540558811833495, '03_rain': -7.755231341347922, '04_rain': -3.4205919985418296, '05_rain': 4.8691369115380825, '06_rain': 4.796

In [9]:
province = input('Please enter PROVINCE:   ')
year = input('Please enter Predict YEAR:   ')
print(' ')

price = {'Rice':10, 'Pineapple':7.5, 'Cassava':1.94, 'Maize':7.35} # thb/kg
cost = {'Rice':3922.32, 'Pineapple':12748.04, 'Cassava':6251.25, 'Maize':4338.38} #thb/rai


rt = df_rt.loc[(df_rt['PROVINCE'] == province) & (df_rt['YEAR'] == year)]
rt = rt.drop(columns = ['PROVINCE' , 'YEAR'])
rt = np.array(rt)

crop1 = final_data[final_data['PROVINCE'] == province]
for i in crop1['PROVINCE_CROP'].unique():
     crop_coef = list(final_dic_province[i].values())
     crop_coef = crop_coef[:-1]
     crop_coef = np.array(crop_coef)
     crop_int = final_dic_province[i]['intercept']
     crop_yield = crop_int + np.sum(crop_coef*rt)
     crop2 = crop1[crop1['PROVINCE_CROP'] == i]
     crop2 = crop2['CROP'].values[0]
     gp = crop_yield*price[crop2] - cost[crop2]
     print(f'Forecast {crop2} Yield = {crop_yield:,.2f} Kg/Rai,  Gross profit = {gp:,.2f} THB/Rai')

Please enter PROVINCE:   Phitsanulok
Please enter Predict YEAR:   2021
 
Forecast Rice Yield = 558.71 Kg/Rai,  Gross profit = 1,664.81 THB/Rai
Forecast Pineapple Yield = 3,379.78 Kg/Rai,  Gross profit = 12,600.33 THB/Rai
Forecast Cassava Yield = 3,332.52 Kg/Rai,  Gross profit = 213.84 THB/Rai
Forecast Maize Yield = 758.14 Kg/Rai,  Gross profit = 1,233.97 THB/Rai
